#🏛️Indian Penal Code Chatbot: Your Legal Assistant 🗣️

In [1]:
import os
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT="basic_project_model_io"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_TRACING_V2=True

In [2]:
from langchain.document_loaders import PyPDFLoader

In [4]:
loader=PyPDFLoader(file_path="data/THE_INDIAN_PENAL_CODE.pdf")
pages=loader.load()
print(len(pages))

112


In [5]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 112
Lines 5724
Words 89077
Charecters 467193


### Step-2: Split the data into Chunks

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

1957


#Step-3: Creating embeddings and Storing in Vector Stores

In [8]:
from langchain_openai import OpenAIEmbeddings
## VectorStores
from langchain_chroma import Chroma

In [9]:
embeddings = OpenAIEmbeddings()

In [10]:
IPC_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="vectorstores/IPC_rules_db"
                          )

#Step-4: Conversation and Retrieval Chain

In [12]:
retriever = IPC_rules_db.as_retriever()

query = """
What is the section related to take part in an unlawful assembly or riot.
"""
result=retriever.invoke(query,top_k=3)
result

[Document(page_content='reason to believe that such riot was likely to be commi tted, or that the unlawful assembly by which such \nriot was committed was likely to be held, shall not use all lawful means in his power to prevent such riot \nor assembly from taking place and for suppr essing and dispersing the same.', metadata={'page': 41, 'source': 'data/THE_INDIAN_PENAL_CODE.pdf'}),
 Document(page_content='5 \n SECTIONS  \n158. Being hired to take part in an unlawful assembly or riot; or to go armed.  \n159. Affray.  \n160. Punishment for committing affray.  \nCHAPTER IX  \nOF OFFENCESBY OR RELATING TO  PUBLIC  SERVANTS  \n161. [Repealed. ]. \n162. [Repealed. ]. \n163. [Repealed. ]. \n164. [Repealed. ].', metadata={'page': 4, 'source': 'data/THE_INDIAN_PENAL_CODE.pdf'}),
 Document(page_content='employed, to join or become members of an unlawful assembly, shall be punished with imprisonment of \neither description for a term which may extend to six mont hs, or with fine, or with both. 

In [15]:
#result[0].page_content
len(result)

4

In [17]:
for i in range(len(result)):
     print(result[i].page_content)

****** reason to believe that such riot was likely to be commi tted, or that the unlawful assembly by which such 
riot was committed was likely to be held, shall not use all lawful means in his power to prevent such riot 
or assembly from taking place and for suppr essing and dispersing the same.
****** 5 
 SECTIONS  
158. Being hired to take part in an unlawful assembly or riot; or to go armed.  
159. Affray.  
160. Punishment for committing affray.  
CHAPTER IX  
OF OFFENCESBY OR RELATING TO  PUBLIC  SERVANTS  
161. [Repealed. ]. 
162. [Repealed. ]. 
163. [Repealed. ]. 
164. [Repealed. ].
****** employed, to join or become members of an unlawful assembly, shall be punished with imprisonment of 
either description for a term which may extend to six mont hs, or with fine, or with both.  
158. Being hired to take part in an unlawful assembly or riot .—Whoever is engaged, or hired, or
****** member of an unlawful assembly, shall be punished with imprisonment of either description for a t

In [18]:

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI

In [19]:
llm=ChatOpenAI(temperature=0)

In [27]:
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory()

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversational_RAG = ConversationalRetrievalChain.from_llm(llm=llm,
                                                     retriever=IPC_rules_db.as_retriever(),
                                                     memory=memory,
                                                     #verbose=True
                                                     )




In [28]:


user_input=input("Your message:")
while user_input!="quit":
    print("AI message ==>", conversational_RAG.invoke(input=user_input))
    user_input=input("Your message: ")
while user_input =="quit":
    memory.clear()
    break